In [1]:
import sys
sys.path.append('../')
# from experiments import utilities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly_express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plot.utils import bar_update_fig, line_update_fig
import pypolo2

width = 1000
height = 800
markersize = 10
linewidth = 4
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
linetype = {'Low':'dot','Medium':'dashdot','High':'longdash'}
# 创建一个包含线型的示例数组
line_dash = ['solid', 'dot', 'dash', 'dashdot', 'longdash']
colors_dict = ['#C74223', '#3D9182', '#245C8F']

In [8]:
# 区域pm2.5平均浓度变化
Setting = pypolo2.utilities.Config()
strategy_name = ["EffectOrientedSelectiveSpray",
                 "EffectOrientedGreedySpray",
                 "MaximumCoverageSpray",
                 "NoSpray"]
comparing_variables = [["teamsize_3","teamsize_5","teamsize_7"],
                       ["numsource_2","numsource_4","numsource_6","numsource_8"],
                       ["change_interval4","change_interval8","change_interval12"]]
seeds = [0,3,7,11,15,18,32,42,50,55,60,66]
team_sizes = [3,5,7]
sourcenums = [2,4,6,8]
change_intervals = [4,8,12]
strategy_meanpm = []
sche_step = 0
adaptive_step = 0
for strategy in strategy_name:
    if strategy == "EffectOrientedGreedySpray":
        sche_step = 8
        adaptive_step = 8
    else:
        sche_step = 18
        adaptive_step = 2
    variable_meanpm = []
    for i in range(3):
        comparing_variable = comparing_variables[0][i]
        team_size = team_sizes[i]
        sourcenum = 3
        time_meanpm = []
        for time_point in range(48):
            seed_meanpm = []
            for seed in seeds:
                pklname = f'SEED_{seed}_X{Setting.grid_x}_Y{Setting.grid_y}_VS{team_size}_TS{Setting.max_num_samples}_SS{sche_step}_AS{adaptive_step}_SN{sourcenum}_RS{Setting.replenish_speed}_WV{Setting.water_volume}'
                pkldir = './outputs/' + strategy + '/' + comparing_variable + '/' + pklname + ".pkl"
                data = pypolo2.experiments.utilities.readpkl(pkldir)
                pm = data['truth_env']
                # 获取特定时间点的效果数据
                seed_meanpm.append(np.mean(pm[time_point]))
            time_meanpm.append(np.mean(seed_meanpm))
        variable_meanpm.append(time_meanpm)
    strategy_meanpm.append(variable_meanpm)
strategy_meanpm = np.array(strategy_meanpm)
# print(strategy_meanpm)



In [20]:
width = 1100
height = 540
linewidth = 3
markersymbol={'FH':4, 'HEUR':17, 'DDRL':2}
markersize = 1
linetype = {'Low':'dot','Medium':'dashdot','High':'solid'}
labels = []
Time_deadline = 48
for i in range(Time_deadline):
    labels.append(int(i*10))

xtitle = 'time(min)'
ytitle = 'Mean pm2.5 concentration (μg/m³)'

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][0][0:Time_deadline], name='EO+3 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[1],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][1][0:Time_deadline], name='EO+5 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[1],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[0][2][0:Time_deadline], name='EO+7 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[1],dash=linetype['High'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][0][0:Time_deadline], name='MC+3 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Low'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][1][0:Time_deadline], name='MC+5 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['Medium'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[2][2][0:Time_deadline], name='MC+7 Sprinkle Truck', mode='lines', line=dict(color=colors_dict[2],dash=linetype['High'], width=linewidth),))
fig.add_trace(go.Scatter(x=labels, y=strategy_meanpm[3][0][0:Time_deadline], name='Non-Spray', marker=dict(size=markersize, symbol=markersymbol['DDRL']),line=dict(color=colors_dict[0], width=linewidth),))


fig = line_update_fig(fig, xtitle, ytitle, width=width, height=height)
fig.update_xaxes(title_text=xtitle, title_font=dict(size=25),tickfont=dict(size=20))
fig.update_yaxes(title_text=ytitle, title_font=dict(size=25),tickfont=dict(size=20))
fig.update_layout(legend=dict(font=dict(size=16))) # name size
fig.show()
filename = './outputs/png/line_averagepollution-teamsize357-linetype.png'
print(filename)
fig.write_image(filename, scale=1)

./outputs/png/line_averagepollution-teamsize357-linetype.png
